<a href="https://colab.research.google.com/github/LeonardoBonanno/SummerResearch2020/blob/master/trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We use this code to generate alternating permutations, alternating sequences of reals, and stochastic tridiagonal matrices.

In [ ]:
def update(x):
    return (2 / math.pi) * math.asin(random.random() * math.cos(math.pi * x/2))

def calculateAcceptanceProb(x_n, x_0):
    alpha = math.cos(math.pi * x_n/ 2) / math.cos(math.pi * x_0/ 2)
    return 1 / (alpha + 1 / alpha)

def generateSuperdiagonal(n, acceptanceProb = None):
    X = [0] * n
    forward = True
    while True:
        X[0] = random.random()
        for i in range(1, n):
            X[i] = update(X[i-1])
        acceptProb = calculateAcceptanceProb(X[n-1], X[0])
        p = random.random()
        if acceptanceProb != None:
            acceptanceProb.append(2 * acceptProb)
        if p < 2 * acceptProb:
            forward = p < acceptProb
            break
    return X, forward

def convertToAlternatingReals(X, forward, n):
    Y = [0] * n
    increment, start = (1, 0) if forward else (-1, n - 1)
    for i in range(n):
        if i % 2 == 0:
            Y[i]= X[start + increment * i]
        else:
            Y[i] = 1 - X[start + increment * i]
    return Y

def convertToPerm(Y, n):
    sigma_prime = list(np.argsort(Y))
    sigma = [0] * n
    for i in range(n):
        sigma[i] = sigma_prime.index(i) + 1
    return sigma

def generateUniformPerm(n, acceptanceProb = None):
    X, forward = generateSuperdiagonal(n, acceptanceProb)
    Y = convertToAlternatingReals(X, forward, n)
    return convertToPerm(Y, n)

First we generate a (uniformly random) stochastic tridiagonal matrix superdiagonal (of length n) like the below. Note how sequential terms always sum to 1. The generateSuperdiagonal function uses a slightly modified version of the algorithm presented in Marchal's paper.

In [ ]:
n = 10
X, forward = generateSuperdiagonal(n)
print(X)


[0.9164303224574469, 0.005891409571226082, 0.709983356869203, 0.06484152862228948, 0.5540709329812937, 0.2992616553605782, 0.339268723900239, 0.25024120285601453, 0.6694943147828405, 0.12384989420472682]


We then apply the bijection from the Diaconis paper to convert this stochastic superdiagonal into an an alternating sequence of reals. 

In [ ]:
Y = convertToAlternatingReals(X, forward, n)
print(Y)

[0.9164303224574469, 0.9941085904287739, 0.709983356869203, 0.9351584713777106, 0.5540709329812937, 0.7007383446394218, 0.339268723900239, 0.7497587971439854, 0.6694943147828405, 0.8761501057952732]


We then convert the alternating sequence of reals into a permutation by sorting.

In [ ]:
sigma = convertToPerm(Y, n)
print(sigma)

[8, 10, 5, 9, 2, 4, 1, 6, 3, 7]


We combine these three steps into one function to generate a uniform alternating permutation as below.

In [ ]:
N = 10
sigma = generateUniformPerm(N)
print(sigma)

[2, 6, 3, 9, 5, 10, 4, 8, 1, 7]


Now that we have code to generate uniform alternating permutations, we show the code to convert from permutations to trees.

In [ ]:
import copy

# generates complement of a permutation defined in Donaghey
def complement(perm):
    n = len(perm)
    sortedElements = sorted(copy.deepcopy(perm))
    complement = [0] * n
    for i in range(n):
        j = sortedElements.index(perm[i])
        complement[i] = sortedElements[n - 1 - j]
    return complement

# returns index of max and minimal elements of a permutation   
def maximalElements(perm):
    minVal = float("inf")
    maxVal = 0
    minIndex = 0
    maxIndex = 0
    for i in range(len(perm)):
        if perm[i] < minVal:
            minVal = perm[i]
            minIndex = i
        if perm[i] > maxVal:
            maxVal = perm[i]
            maxIndex = i
    return maxIndex, minIndex

# converts regular permutation to its tree permutation form recursively
def convertToTreePerm(perm):
    if len(perm) <= 1:
        return perm
    maxIndex, minIndex = maximalElements(perm)
    if minIndex < maxIndex:
        perm = complement(perm)
        minIndex = maxIndex
    return convertToTreePerm(perm[0:minIndex]) + [perm[minIndex]] + convertToTreePerm(perm[minIndex + 1:])

# returns index and value of minimal element in a permutation
def minimalElement(perm):
    minVal = float("inf")
    minIndex = 0
    for i in range(len(perm)):
        if perm[i] < minVal:
            minVal = perm[i]
            minIndex = i
    return minIndex, minVal

# returns index and value of maximal element in a permutation
def maximalElement(perm):
    maxVal = 0
    maxIndex = 0
    for i in range(len(perm)):
        if perm[i] > maxVal:
            maxVal = perm[i]
            maxIndex = i
    return maxIndex, maxVal

# calculate the number of cherries in a regular permutation
def calculateCherries(perm):
    convertedPerm = convertToTreePerm(perm)
    cherries = [0]
    calculateCherriesHelper(convertedPerm, cherries)
    return cherries[0]

# helper function to calculate number of cherries in a permutation
def calculateCherriesHelper(perm, cherries):
    n = len(perm)
    if n == 1:
        cherries[0] += 1
    minIndex, minVal = minimalElement(perm)
    if minIndex != 0:
        calculateCherriesHelper(perm[:minIndex], cherries)
    if minIndex != n - 1:
        calculateCherriesHelper(perm[minIndex + 1:], cherries)

Most of the above functions are just helper functions and are not too useful so we only look at the convertToTreePerm and calculateCherries functions. We first get a permutation from the prior code.

In [ ]:
n = 10
sigma = generateUniformPerm(n)

The convertToTreePerm function performs the first part of the bijection presented in Donaghey's paper. It returns a modified permutation (using the bracketing and nestling process described), from which the final tree can be easily recovered. I call these modified permutations "tree permutations".

In [ ]:
sigma_prime = convertToTreePerm(sigma)
print(sigma_prime)

[10, 9, 4, 2, 6, 3, 1, 8, 5, 7]


We can also recover the number of cherries that the corresponding tree of a permutation has using the below function.

In [ ]:
print(calculateCherries(sigma))

3


This is the code we will use to generate random ranked unlabeled trees and print out their ordered matching representations.

In [ ]:
# generate uniformly random tree perm in order easiest to print
def generateUniformTreePerm(n):
    return complement(convertToTreePerm(generateUniformPerm(n)))

# helper sorting function
def mySort(tuple):
    return tuple[2]

# tree class to help print out
class Tree(object):
    def __init__(self, data):
        self.data = data
        self.right = None
        self.left = None

    def getRight(self):
        return self.right

    def getLeft(self):
        return self.left

    def setRight(self, rightTree):
        self.right = rightTree

    def setLeft(self, leftTree):
        self.left = leftTree

    def calculateCherriesHelper(self, cherries):
        leftEmpty = self.left is None
        rightEmpty = self.right is None
        if leftEmpty and rightEmpty:
            cherries[0] += 1
        if not leftEmpty:
            self.left.calculateCherriesHelper(cherries)
        if not rightEmpty:
            self.right.calculateCherriesHelper(cherries)

    def calculateCherries(self):
        cherries = [0]
        self.calculateCherriesHelper(cherries)
        return cherries[0]
    
    # recursively traverses tree   
    def traverse(self, representation):
        info = [0,0]
        if self.left is not None:
            info[0] = self.left.data
            self.left.traverse(representation)
        if self.right is not None:
            info[1] = self.right.data
            self.right.traverse(representation)
        info = sorted(info)
        info.append(self.data)
        info = tuple(info)
        representation.append(info)

    def getTree(self):
        representation = []
        self.traverse(representation)
        representation.sort(key = mySort)
        return representation
        
    def __str__(self):
        representation = []
        self.traverse(representation)
        representation.sort(key = mySort)
        treeAsString = ""
        for i in range(len(representation)):
            treeAsString += str((representation[i][0], representation[i][1])) + "^{" + str(representation[i][2]) + "}" + "\n"
        return treeAsString

# build a tree from a proper tree permutation, you can print right and left subtrees as well
def buildTree(treePerm):
    n = len(treePerm)
    if n == 1:
        return Tree(treePerm[0])
    maxIndex, maxVal = maximalElement(treePerm)
    newTree = Tree(maxVal)
    if maxIndex != 0:
        newTree.setRight(buildTree(treePerm[:maxIndex]))
    if maxIndex != n - 1:
        newTree.setLeft(buildTree(treePerm[maxIndex + 1:]))
    return newTree

This funtion combines the alternating permutation and tree conversion functions we looked at earlier to generate a uniformly random tree permutation. As you can see in the code, we use the complement function before returning the final result because we build the tree top down (we want to have n at the top rather than 1).

In [ ]:
n = 10
treePerm = generateUniformTreePerm(n)
print(treePerm)

[1, 2, 4, 3, 5, 6, 10, 7, 8, 9]


We can then use the buildTree function to convert the above tree perm into a Tree from the Tree class above. The tree class has the print trait implemented so the trees can be printed out in the ordered matching representation. This method is very hacky and I imagine there is a better way to do it.

In [ ]:
tree = buildTree(treePerm)
print(tree)

(0, 0)^{1}
(0, 1)^{2}
(0, 0)^{3}
(2, 3)^{4}
(0, 4)^{5}
(0, 5)^{6}
(0, 0)^{7}
(0, 7)^{8}
(0, 8)^{9}
(6, 9)^{10}



We can also get the number of cherries in the tree using the calculateCherries method.

In [ ]:
print(tree.calculateCherries())

3


Now we have code to generate alternating permutations and convert them to trees and print them, so we investigatae some markov chains on alternating permutations. The first being the alternating permutations random transpositions chain, where at each stage, two (non-adjacent) elements are flipped (if allowed).

In [ ]:
import copy
import random

# helper function for generating two numbers that differ by more than one
def generateIJ(n):
    while True:
            [i, j] = sorted(random.sample(range(n), 2))
            if abs(i - j) > 1:
                break
    return i, j

# swaps two elements if tajima allows for it
def updateIJ(currentPerm, i, j, tajima):
    if tajima:
        otherPerm = copy.deepcopy(currentPerm)
        sigma_i = currentPerm[j]
        sigma_j = currentPerm[i]
        otherPerm[i] = currentPerm[j]
        otherPerm[j] = currentPerm[i]
        p = random.random()
        if p < min(1, 2 ** (calculateCherries(currentPerm) - calculateCherries(otherPerm))):
            currentPerm[i] = sigma_i
            currentPerm[j] = sigma_j
    else:
        temp = currentPerm[i]
        currentPerm[i] = currentPerm[j]
        currentPerm[j] = temp

# helper function, whether swapping of two elements is allowed
def canUpdate(currentPerm, n, i, j):
    isFirst = (i == 0)
    isLast = (j == n - 1)
    sigma_i = currentPerm[i]
    sigma_j = currentPerm[j]
    sigma_ip1 = currentPerm[i + 1]
    sigma_jm1 = currentPerm[j - 1]
    firstDecreasing = (sigma_i > sigma_ip1)
    secondDecreasing = (sigma_jm1 < sigma_j) 
    if not isFirst:
        if (currentPerm[i - 1] < sigma_j) != firstDecreasing:
            return False
    if not isLast:
        if (sigma_i > currentPerm[j + 1]) != secondDecreasing:
            return False
    if (sigma_j > sigma_ip1) != firstDecreasing:
        return False
    if (sigma_jm1 < sigma_i) != secondDecreasing:
        return False
    return True

# implements MCMC markovian update 
def updateAP(currentPerm, n, tajima):
    i, j = generateIJ(n) 
    if canUpdate(currentPerm, n, i, j):
        updateIJ(currentPerm, i, j, tajima)

def difference(first, second, n):
    total = 0
    for i in range(n):
        if first[i] != second[i]:
            total += 1
    return total
    
# alternating permutation MCMC with element swapping
# tajima for whether want to metropolize the chain to converge to tajima
# printTrees for whether want to print trees as we go
def alternatingPermMCMC(startingPerm, samples, tajima, printTrees, frequencies):
    n = len(startingPerm)
    currentPerm = startingPerm
    sampDic = {}
    total = 0
    currentRep = [(0,0,0)] * n
    for _ in range(samples):
        c = calculateCherries(currentPerm)
        tree = buildTree(complement(convertToTreePerm(currentPerm)))
        newRep = tree.getTree()
        total += difference(newRep, currentRep, n)
        currentRep = newRep
        if printTrees:
            print(tree)
        if frequencies:
          if tuple(currentPerm) in sampDic:
            sampDic[tuple(currentPerm)] += 1
          else:
            sampDic[tuple(currentPerm)] = 1
        updateAP(currentPerm, n, tajima)
    print("The average number of pairs that were changed is:", (total - 1)/samples)
    if frequencies:
      for entry in sampDic:
        print(entry, "has frequency", sampDic[entry] * 100 / samples, "%")



We first look at a single step for this chain. The chain normally has equilibrium distribution as uniform, but I have implemented a feature to metropolize the chain to converge to the tajima. So throughout the code, if you want the updates to be the tajima updates, set the tajima flag to true. The updateAP function performs the update.

In [ ]:
N = 10
tajima = False

X = generateUniformPerm(N)
print(buildTree(complement(convertToTreePerm(X))))

updateAP(X, N, tajima)

print(buildTree(complement(convertToTreePerm(X))))

(0, 0)^{1}
(0, 0)^{2}
(0, 0)^{3}
(0, 2)^{4}
(0, 3)^{5}
(0, 0)^{6}
(4, 5)^{7}
(1, 7)^{8}
(6, 8)^{9}
(0, 9)^{10}

(0, 0)^{1}
(0, 0)^{2}
(0, 0)^{3}
(0, 2)^{4}
(1, 4)^{5}
(0, 0)^{6}
(3, 5)^{7}
(0, 7)^{8}
(6, 8)^{9}
(0, 9)^{10}



You can see above that many of the pairs can be changed in just a single update. We can now run this chain using the below function. Set the tajima flag if you want the chain to converge to the tajima distribution and the printTrees function if you want the trees to be printed out as the chain is running (if you do this, set the samples to be lower). Set the frequencies flag if to see the freuqency of each permutation. 

In [ ]:
N = 10
samples = 10
tajima = False
printTrees = True
frequencies = False

alternatingPermMCMC(generateUniformPerm(N), samples, tajima, printTrees, frequencies)

We often need to calculate the Euler-Zigzag numbers. Here is the code to do so.

In [ ]:
# returns list with first n Euler-Zigzag Numbers (E_0 - E_n)

def comb(n, k):
  return int(math.factorial(n) / (math.factorial(k) * math.factorial(n - k)))
def calculateEulerZigzagNumbers(n):
    E = [0] * (n + 1)
    E[0] = 1
    E[1] = 1
    for i in range(2, n + 1):
        calculateEulerZigzagNumber(i, E)
    return E

def calculateNthEulerZigzagNumber(n):
    return calculateEulerZigzagNumbers(n)[n]

# helper function to calculate a single euler-zigzag number
def calculateEulerZigzagNumber(i, E):
    E_i= 0
    for k in range(1, i, 2):
        combin = comb(i - 1, k)
        E_i += combin * E[k] * E[i - 1 - k]
    E[i] = E_i

To calculate the nth Euler-Zigzag number we need all of the lower ones.

In [ ]:
n = 20
E_n = int(calculateNthEulerZigzagNumber(n))
print(E_n)

370371188237525


In Diaconis's paper on stochastic tridiagonal matrices, they present an algorithm for generating an exactly uniform alternating permutation recursively. I present the implementation of this algorithm below. Naively, it has O(n^3) time complexity, due to the complexity of calculating the first n Euler-Zigzag numbers but using the asymptotic expansion, we can get nearly identical results, allowing this to be a faster algorithm, although it is still slow because of calculating the factorials.

In [ ]:
import math
import numpy as np
import random

rng = np.random.default_rng()

def recursiveExactSampling(n):
    sigma = [0] * n
    numbers = [i for i in range(1, n + 1)]
    return samplingHelper(numbers, True)

def samplingHelper(numbers, forward):
    n = len(numbers)
    if n == 0:
        return []
    if n == 1:
        return [numbers[0]]
    k = getK(n, forward)
    if forward:
        lowerNumbers = sorted(random.sample(numbers[0:n-1], k - 1)) if k > 1 else []
        upperNumbers = sorted(list(set(numbers[0:n - 1]).difference(set(lowerNumbers))))
        return samplingHelper(lowerNumbers, forward) + [numbers[n-1]] + samplingHelper(upperNumbers, not forward)
    else:
        lowerNumbers = sorted(random.sample(numbers[1:], k - 1)) if k > 1 else []
        upperNumbers = sorted(list(set(numbers[1:]).difference(set(lowerNumbers))))
        return samplingHelper(lowerNumbers, forward) + [numbers[0]] + samplingHelper(upperNumbers, not forward)
        

def getK(n, forward):
    E = calculateEulerZigzagNumbers(n)
    values = list(range(1, n + 1, 2))
    p = [comb(n - 1, k - 1) * E[k - 1] * E[n - k] / E[n] for k in values]
    k = values[list(rng.multinomial(1, p)).index(1)]
    return k

Use the recursive exact sampling function to recursively sample a uniform alternating permutation.

In [ ]:
n = 10
print(recursiveExactSampling(n))

[6, 4, 9, 1, 5, 3, 8, 2, 10, 7]
